<link href="https://fonts.googleapis.com/css2?family=Inter:wght@400&display=swap" rel="stylesheet">
<div style="font-family: 'Inter'; font-size: 24px; color: #749857;"><B>🍏 Show Up</b> for Health</div>
<div style="font-family: 'Inter'; font-size: 14px; color: #7a7979;"><B>Predicting Missed Apppointment in Primary Care</b> - Brompton Health PCN</div>

[janduplessis883](https://github.com/janduplessis883)<BR>
[AlexAlexRose](https://github.com/AlexAlexRose)<BR>
[FabySp](https://github.com/FabySp)<BR>
[mogleyza](https://github.com/mogleyza)

# FabySp | OneHotEncode Rota type, Ethnicity category & Language


### Importing Libraries

In [1]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

### Params

In [2]:
RAW_DATA = '~/code/janduplessis883/data-showup/data/output-data/training_data_no_features.csv'
WEATHER_DATA = '~/code/janduplessis883/data-showup/data/weather/weather.csv'
IMD_DATA = '~/code/janduplessis883/data-showup/data/imd-master/imd_master.csv'

OUTPUT_DATA = '~/code/janduplessis883/data-showup/data/output-data/'

In [ ]:
# # Adding the function to merge the entire dataset available with weather dataset

# def make_appointments_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
#     all_surgery_list = []
    
#     for surgery in surgery_list:
#         surgery_appointment_list = []
        
#         for i in range(1,10):
#             app_path = f'{RAW_DATA}{surgery}/{surgery}_app{i}.csv'
#             single_app_file = pd.read_csv(app_path)
#             surgery_appointment_list.append(single_app_file)
            
#         surgery_apps_df = pd.concat(surgery_appointment_list, ignore_index=True)
#         #print(f'The total rows of the {surgery} surgery is {surgery_apps_df.shape[0]}')
        
#         all_surgery_list.append(surgery_apps_df)
        
#     full_list = pd.concat(all_surgery_list, ignore_index=True)
#     return full_list
# make_appointments_register()

In [3]:
df = pd.read_csv(RAW_DATA)

In [4]:
df.head()

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
1,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
2,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
3,12-May-22,05-Jul-22,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,Female,2014-01-07 00:00:00,Other White,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996
4,17-May-22,05-Jul-22,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,Male,2017-03-06 00:00:00,Indian or British Indian,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309


In [5]:
import re

def extract_rota_type(text):
    # HOW TO APPLY IT
    # Apply extract_role function and overwrite Rota type column
    # full_appointments['Rota type'] = full_appointments['Rota type'].apply(extract_rota_type)     
    role_map = {
    'GP': ['GP', 'Registrar', 'Urgent', 'Telephone', '111', 'FY2', 'F2', 'Extended Hours', 'GP Clinic', 'Session'],
    'Nurse': ['Nurse', 'Nurse Practitioner'], 
    'HCA': ['HCA','Health Care Assistant', 'Phlebotomy'],
    'ARRS': ['Pharmacist', 'Paramedic', 'Physiotherapist', 'Physicians Associate', 'ARRS', 'PCN'],
    }

    for role, patterns in role_map.items():
        for pattern in patterns:
            if re.search(pattern, text):
                return role
    return 'DROP'

In [6]:
def map_rota_type(df):
    df['Rota type'] = df['Rota type'].map(extract_rota_type)
    return(df)

In [9]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encode_columns(df, columns_to_encode=['Rota type', 'Ethnicity category', 'Language']):
    """
    Perform One-Hot Encoding on specified columns in a DataFrame.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        columns_to_encode (list): List of column names to OneHot Encode.

    Returns:
        pd.DataFrame: A new DataFrame with OneHot Encoded columns and original columns dropped.
    """
    # Create an instance of OneHotEncoder
    encoder = OneHotEncoder()

    # Initialize an empty DataFrame to store the encoded columns
    encoded_df = pd.DataFrame()

    # Loop over the columns to encode
    for column in columns_to_encode:
        # Fit the encoder to the current column and transform the data
        encoded_data = encoder.fit_transform(df[[column]])

        # Convert the result to a DataFrame and add it to the encoded DataFrame
        encoded_df = pd.concat([
            encoded_df, 
            pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out([column]))
        ], axis=1)

    # Concatenate the encoded DataFrame with the original DataFrame, dropping the original columns
    df = pd.concat([df, encoded_df], axis=1)
    df.drop(columns=columns_to_encode, inplace=True)

    return df

one_hot_encode_columns(df)


,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg,Rota type_ADMIN,Rota type_ADMIN SESSION,Rota type_Care Coordinator,Rota type_Clinical Research,Rota type_DES Extended Hours,Rota type_Diabetes VGC,Rota type_Duty Doctor,Rota type_Earls Court GP,Rota type_Earls Court Registrar,Rota type_Earls Court Surgery ANP,Rota type_Earls Court Surgery Extended Hours GP,Rota type_Earls Court Surgery Extended Hours Nurse,Rota type_Earls Court Surgery GP,Rota type_Earls Court Surgery HCA,Rota type_Earls Court Surgery Nurse,Rota type_Earls Court Surgery Pharmacist,Rota type_Earls Court Surgery Registrar,Rota type_Extended Hours DES,Rota type_Flu,Rota type_Flu Clinic,Rota type_GP Clinic,Rota type_GP Registrar Session,Rota type_GP Session,Rota type_Group Consultation,Rota type_HCA Clinic,Rota type_HCA Session,Rota type_HCA Sessions,Rota type_HCA session,Rota type_Handover - DO NOT BOOK,Rota type_Health Check,Rota type_Health Check Session Extra,Rota type_Health Partners at VM GP,Rota type_Health Partners at VM HCA,Rota type_Health Partners at VM Nurse,Rota type_Health Partners at VM Pharmacist,Rota type_Health Partners at VM Registrar,Rota type_Hub Session,Rota type_KMC GP,Rota type_KMC Nurse,Rota type_KMC Nurse Practitioner,Rota type_KMC Paramedic,Rota type_KMC Pharmacist,Rota type_KMC Registrar,Rota type_Knightsbridge Extended Hours GP,Rota type_Knightsbridge Extended Hours Nurse,Rota type_Locum GP Session,Rota type_MCMW Frailty MDT,Rota type_MCMW Home Visits,Rota type_Med Student Session,Rota type_Mental Health,Rota type_Minor Surgery Clinic,Rota type_Morning Clinic,Rota type_My Care My Way,Rota type_NHS 111,Rota type_NHS 111 Bookings,Rota type_NURSING HOME,Rota type_Nornal Clinic,Rota type_North Kensington Extended Hours GP,Rota type_North Kensington Extended Hours Nurse,Rota type_Nurse Clinic,Rota type_Nurse Earls Court,Rota type_Nurse Triage,Rota type_Nurse Violet Melchett,Rota type_PCN,Rota type_Phlebotomy clinic,Rota type_Physiotherapy,Rota type_Polypharmacy Clinic,Rota type_Postnatal Clinic,Rota type_RANDOM FLU CLINIC,Rota type_Redcliffe Extended Hours GP,Rota type_Redcliffe Extended Hours HCA,Rota type_Redcliffe Extended Hours Nurse,Rota type_Redcliffe Extended Hours Pharmacist,Rota type_Redcliffe F2,Rota type_Redcliffe FY2,Rota type_Redcliffe GP,Rota type_Redcliffe HCA,Rota type_Redcliffe Nurse,Rota type_Redcliffe Pharmacist,Rota type_Redcliffe Registrar,Rota type_SESSION PLS BOOK PATIENTS IN,Rota type_Session,Rota type_Smear Clinic,Rota type_Smoking Advisor,Rota type_Smoking Cessation,Rota type_Special,Rota type_Stanhope Mews Extended Hours GP,Rota type_Stanhope Mews Extended Hours Nurse,Rota type_Stanhope Mews GP,Rota type_Stanhope Mews Nurse,Rota type_Stanhope Mews Nurse Practitioner,Rota type_Stanhope Mews Pharmacist,Rota type_Stanhope Mews Pharmacist ARRS,Rota type_Stanhope Mews Registrar,Rota type_Stanhope Mews Staff,Rota type_Telephone Clinic,Rota type_Telephone Triage,Rota type_The Chelsea Practice GP,Rota type_The Chelsea Practice HCA,Rota type_The Chelsea Practice Nurse,Rota type_The Chelsea Practice Nurse Practitioner,Rota type_The Chelsea Practice Pharmacist,Rota type_The Chelsea Practice Pharmacist ARRS,Rota type_The Chelsea Practice Physicians Associate,Rota type_The Chelsea Practice Registrar,Rota type_The Good Practice Extended Hours GP,Rota type_The Good Practice GP,Rota type_The Good Practice HCA,Rota type_The Good Practice Nurse,Rota type_The Good Practice Nurse Practitioner,Rota type_The Good Practice Pharmacist,Rota type_The Good Practice Physicians Associate,Rota type_The Good Practice Registrar,Rota type_Tutorial Session,Rota type_Urgent Clinics,Rota type_Violet Melchett GP,Rota type_Violet Melchett HCA,Rota type_Violet Melchett Nurse,Rota type_Violet Me

In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# def one_hot_encode_columns(df, columns_to_encode):
#     """
#     Perform One-Hot Encoding on specified columns in a DataFrame.

#     Parameters:
#         df (pd.DataFrame): The input DataFrame.
#         columns_to_encode (list): List of column names to One-Hot Encode.

#     Returns:
#         pd.DataFrame: A new DataFrame with One-Hot Encoded columns and original columns dropped.
#     """
#     # Create an instance of OneHotEncoder
#     encoder = OneHotEncoder()

#     # Fit the encoder to the specified categorical columns and transform the data
#     encoded_data = encoder.fit_transform(df[columns_to_encode])

#     # Convert the result to a DataFrame
#     encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(columns_to_encode))

#     # Concatenate the encoded DataFrame with the original DataFrame, dropping the original columns
#     df = pd.concat([df, encoded_df], axis=1)
#     df.drop(columns=columns_to_encode, inplace=True)

#     return df

# # Columns to One-Hot Encode
# columns_to_encode = ['Rota type', 'Ethnicity category', 'Language']

# # Call the function to perform One-Hot Encoding
# df_encoded = one_hot_encode_columns(df, columns_to_encode)

# df_encoded.head()


In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# # Create an instance of OneHotEncoder
# encoder = OneHotEncoder()

# # Fit the encoder to the categorical columns and transform the data
# encoded_data = encoder.fit_transform(df[['Rota type', 'Ethnicity category', 'Language']])

# # Convert the result to a DataFrame
# encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['Rota type', 'Ethnicity category', 'Language']))

# # Concatenate the encoded DataFrame with the original DataFrame, dropping the original columns
# df = pd.concat([df, encoded_df], axis=1)
# df.drop(columns=['Rota type', 'Ethnicity category', 'Language'], inplace=True)

# df.head()


In [ ]:
import math

def haversine_distance(surgery_prefix, lat2, lon2):
    R = 6371.0  # Radius of the Earth in kilometers

    if surgery_prefix == 'ECS':
        lat1, lon1 = 51.488721, -0.191873
    elif surgery_prefix == 'SMW':
        lat1, lon1 = 51.494474, -0.181931
    elif surgery_prefix == 'TCP':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'HPVM':
        lat1, lon1 = 51.48459, -0.171887
    elif surgery_prefix == 'KMC':
        lat1, lon1 = 51.49807, -0.159918
    elif surgery_prefix == 'TGP':
        lat1, lon1 = 51.482652, -0.178066


    # Convert degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Differences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance  # in kilometers

In [ ]:
def make_global_disease_register(surgery_list = ['ECS', 'TCP', 'TGP', 'SMW', 'KMC', 'HPVM']):
    print('=== Preparing Global Disease Register + IMD2023 info ======================================')

    disease_register = []
    for surgery in surgery_list:
        register_path = f'{RAW_DATA}{surgery}/{surgery}'

        idnhs = pd.read_excel(f'{register_path}_NHS_PTID.xlsx', dtype='str')
        idnhs.dropna(inplace=True)
        frail = pd.read_csv(f'{register_path}_FRAILTY.csv', dtype='str')
        dep = pd.read_csv(f'{register_path}_DEPRESSION.csv', dtype='str')
        obesity = pd.read_csv(f'{register_path}_OBESITY.csv', dtype='str')
        chd = pd.read_csv(f'{register_path}_IHD.csv', dtype='str')
        dm = pd.read_csv(f'{register_path}_DM.csv', dtype='str')
        hpt = pd.read_csv(f'{register_path}_HPT.csv', dtype='str')
        ndhg = pd.read_csv(f'{register_path}_NDHG.csv', dtype='str')
        smi = pd.read_csv(f'{register_path}_SMI.csv', dtype='str')

        ptid = idnhs.merge(frail, how='left', on='NHS number')
        ptid = ptid.drop(columns='NHS number')

        register = (ptid.merge(dep, how='left', on='Patient ID')
                    .merge(obesity, how='left', on='Patient ID')
                    .merge(chd, how='left', on='Patient ID')
                    .merge(dm, how='left', on='Patient ID')
                    .merge(hpt, how='left', on='Patient ID')
                    .merge(ndhg, how='left', on='Patient ID')
                    .merge(smi, how='left', on='Patient ID')
                    .fillna(0)
                    )
        print(f'💊 {surgery} Disease Register completed')
        # Add IMD and distance from station
        imd = pd.read_csv(IMD_DATA)

        full_register = register.merge(imd, how='left', on='Postcode')
        print(f'🔸 {surgery} IMD2023')
        full_register['distance_from_surg'] = full_register.apply(lambda row: haversine_distance(surgery, row['Latitude'], row['Longitude']), axis=1)
        disease_register.append(full_register)

    global_register = pd.concat(disease_register, axis=0, ignore_index=True)
    print(f"🦠 Concat Registers into ONE REGISTER")
    
    global_register.dropna(inplace=True)
    print(f'❌ Dropped NaN')
    
    output_path = f'{OUTPUT_DATA}global_disease_register.csv'
    global_register.to_csv(output_path, index=False)
    print(f'✅ Global Disease Register Saved to output-data: {global_register.shape}')
    print()
    return global_register

In [ ]:
disease_register = make_global_disease_register()